In [19]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
loans = pd.read_csv('data/lending-club-data.csv')
train_idx = pd.read_json('data/module-5-assignment-1-train-idx.json')[0]
validation_idx = pd.read_json('data/module-5-assignment-1-validation-idx.json')[0]
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(columns=['bad_loans'])
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]
train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
def one_hot_encoding(df, target):
    categorical_features = []
    for feature_name, feature_type in zip(df.columns, df.dtypes):
        if feature_type is np.dtype('O'):
            categorical_features.append(feature_name)
            
    for feature_name in categorical_features:
        df2 = pd.get_dummies(df[feature_name],prefix=feature_name)
        df = df.join(df2).drop([feature_name], axis=1)

    t = df[target]
    return (df.drop([target], axis=1), t)


train_data, train_target = one_hot_encoding(train_data, target)
validation_data, validation_target = one_hot_encoding(validation_data, target)

In [25]:
from sklearn import tree


decision_tree_model = tree.DecisionTreeClassifier(max_depth=6)
decision_tree_model.fit(train_data, train_target)
# tree.plot_tree(decision_tree_model)
small_model = tree.DecisionTreeClassifier(max_depth=2)
small_model.fit(train_data, train_target)
big_model = tree.DecisionTreeClassifier(max_depth=10)
big_model.fit(train_data, train_target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [4]:
validation_safe_loans = validation_data[validation_target == 1]
validation_risky_loans = validation_data[validation_target == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,grade_A,grade_B,grade_C,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
19,0,11,11.18,1,1,82.4,0.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
79,0,10,16.85,1,1,96.4,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
24,0,3,13.97,0,1,59.5,0.0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
41,0,11,16.33,1,1,62.1,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
decision_tree_model.predict(sample_validation_data)

array([ 1, -1, -1,  1], dtype=int64)

In [6]:
decision_tree_model.predict_proba(sample_validation_data)

array([[0.34156543, 0.65843457],
       [0.53630646, 0.46369354],
       [0.64750958, 0.35249042],
       [0.20789474, 0.79210526]])

In [136]:
small_model.predict_proba(sample_validation_data)

array([[0.41896585, 0.58103415],
       [0.59255339, 0.40744661],
       [0.59255339, 0.40744661],
       [0.23120112, 0.76879888]])

In [26]:
print(decision_tree_model.score(train_data, train_target))
print(decision_tree_model.score(validation_data, validation_target))
print(small_model.score(train_data, train_target))
print(small_model.score(validation_data, validation_target))
print(big_model.score(train_data, train_target))
print(big_model.score(validation_data, validation_target))

0.6405276165914464
0.6361482119775959
0.6135020416935311
0.6193451098664369
0.6637921770900495
0.6261309780267126


In [27]:
# Assume a cost of $10,000 per false negative.
# Assume a cost of $20,000 per false positive.
cost = 0
for predict, t in zip(decision_tree_model.predict(validation_data), validation_target):
    if predict == -1 and t == 1:
        cost += 10000
    elif predict == 1 and t == -1:
        cost += 20000
cost

50390000

In [7]:
!pip install graphviz